<a href="https://colab.research.google.com/github/mhkirali/numpy_test1/blob/main/999_Down_Trendline_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['close'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'close').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]

    hh_pairs_next = argrelextrema(df_next['close'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    # Iterate over each next highest point and calculate the slope
    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['close'].values[0], next_hp['close']])
        # Store the slope
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        # Find the minimum slope and its corresponding intercept
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['close'] > df['trend']
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


Titles = ['Hisse Adı', 'Son Fiyat' ,'Trend Değeri','Yüzde', 'Yakınlık Durumu','Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)
for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=Interval.in_4_hour, n_bars=500)
        data = data.reset_index()
        Trend_line = Down_Trend_Line(data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()
        Last_Price = Signals.loc[order-1, 'close']
        Last_Trend = Signals.loc[order-1, 'close']
        Last_Perc = ((Signals.loc[order-1,'trend']- Signals.loc[order-1,'close'])/Signals.loc[order-1,'trend'])*100
        Close_Status = False
        Break_Status = False
        if Last_Perc<=5 and Last_Trend>Last_Price:
            Close_Status  = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1,'Entry']
        if Entry==True and Last_Entry==True:
              Break_Status = True

        L1 = [Hisseler[i] ,round(Last_Price,2), round(Last_Trend,2), round(Last_Perc,2),Close_Status,Break_Status]
        print(L1)
        df_down_trend.loc[len(df_down_trend)] = L1
    except:
        pass

df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)]
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)]

print('Yakın Olan Hisseler')
print(df_close.to_string())
print('Kırılma Gerçekleşmiş Olanlar')
print(df_Breakout.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-zzj6cpb3
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-zzj6cpb3
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Using cached tradingview_screener-2.4.1-py3-none-any.whl (66 kB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=043367e6e14d4fe9932492e29dcf38eb0fe37dc36be71d0045db392ea6f2062e
  Stored in directory: /tmp/pip-ephem-wheel-cache-s03z5v_v/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency re

['A1CAP', 27.84, 27.84, 31.94, False, False]
['ACSEL', 145.0, 145.0, 18.22, False, False]
['ADEL', 624.0, 624.0, 20.45, False, False]
['ADESE', 2.31, 2.31, 9.87, False, False]
['ADGYO', 27.7, 27.7, 30.11, False, False]
['AEFES', 202.5, 202.5, 1.36, False, False]
['AFYON', 13.13, 13.13, 13.4, False, False]
['AGROT', 18.57, 18.57, 46.58, False, False]
['AGYO', 8.78, 8.78, 8.43, False, False]
['AKBNK', 68.5, 68.5, -8.47, False, False]
['AKCNS', 139.8, 139.8, 7.31, False, False]
['AKENR', 15.94, 15.94, 16.26, False, False]
['AKFGY', 2.33, 2.33, 7.55, False, False]
['AKFYE', 27.06, 27.06, 1.88, False, False]
['AKGRT', 7.5, 7.5, 6.84, False, False]
['AKMGY', 297.75, 297.75, 20.1, False, False]
['AKSA', 120.4, 120.4, -1.09, False, True]
['AKSGY', 17.99, 17.99, 3.29, False, False]
['AKSUE', 13.17, 13.17, 17.03, False, False]
['ALARK', 122.3, 122.3, -2.88, False, True]
['ALBRK', 6.51, 6.51, -4.16, False, False]
['ALCAR', 1232.0, 1232.0, 17.1, False, False]
['ALCTL', 110.4, 110.4, 13.49, False, 

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['MAKTK', 7.29, 7.29, 3.16, False, False]
['MANAS', 10.23, 10.23, 11.19, False, False]
['MARBL', 14.94, 14.94, 13.14, False, False]
['MARKA', 83.7, 83.7, 6.7, False, False]
['MARTI', 3.77, 3.77, 8.35, False, False]
['MEDTR', 32.54, 32.54, 10.91, False, False]
['MEGAP', 10.19, 10.19, -62.58, False, False]
['MEGMT', 36.28, 36.28, 16.12, False, False]
['MEKAG', 61.8, 61.8, 27.31, False, False]
['MEPET', 12.28, 12.28, 13.76, False, False]
['MERCN', 11.87, 11.87, 23.69, False, False]
['MERIT', 154.2, 154.2, -4.15, False, True]
['METRO', 2.86, 2.86, 3.57, False, False]
['METUR', 15.42, 15.42, -8.25, False, True]
['MHRGY', 4.77, 4.77, 5.05, False, False]
['MIATK', 50.4, 50.4, 13.25, False, False]
['MIPAZ', 81.1, 81.1, 8.82, False, False]
['MMCAS', 24.5, 24.5, -31.75, False, False]
['MNDRS', 14.37, 14.37, 5.27, False, False]
['MNDTR', 6.7, 6.7, 8.42, False, False]
['MOBTL', 5.87, 5.87, 16.41, False, False]
['MOGAN', 13.41, 13.41, 11.62, False, False]
['MRGYO', 5.49, 5.49, 40.69, False, False]


In [1]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['close'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'close').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]

    hh_pairs_next = argrelextrema(df_next['close'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    # Iterate over each next highest point and calculate the slope
    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['close'].values[0], next_hp['close']])
        # Store the slope
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        # Find the minimum slope and its corresponding intercept
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['close'] > df['trend']
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


Titles = ['Hisse Adı', 'Son Fiyat' ,'Trend Değeri','Yüzde', 'Yakınlık Durumu','Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)
for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=Interval.in_4_hour, n_bars=500)
        data = data.reset_index()
        Trend_line = Down_Trend_Line(data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()
        Last_Price = Signals.loc[order-1, 'close']
        Last_Trend = Signals.loc[order-1, 'close']
        Last_Perc = ((Signals.loc[order-1,'trend']- Signals.loc[order-1,'close'])/Signals.loc[order-1,'trend'])*100
        Close_Status = False
        Break_Status = False
        if Last_Perc<=5 and Last_Trend>Last_Price:
            Close_Status  = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1,'Entry']
        if Entry==True and Last_Entry==True:
              Break_Status = True

        L1 = [Hisseler[i] ,round(Last_Price,2), round(Last_Trend,2), round(Last_Perc,2),Close_Status,Break_Status]
        print(L1)
        df_down_trend.loc[len(df_down_trend)] = L1
    except:
        pass

df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)]
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)]

print('Yakın Olan Hisseler')
print(df_close.to_string())
print('Kırılma Gerçekleşmiş Olanlar')
print(df_Breakout.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-n69uf0my
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-n69uf0my
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.6 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=254447d0a3025e8e60fee191032e690387211dac1350e840453238edf877e124
  Stored in directory: /tmp/pip-ephem-wheel-cache-1zwtdgw_/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'tradingview_screener'